<!--NAVIGATION-->
< [Condition 1](Condition_1.ipynb) | [Contenus](Contents.ipynb) | [Condition 3](Condition_3.ipynb) >

# 4. Condition 2: Pas d’industrie, de mine ou de décharge à moins de 300 mètres des puits
## 4.1 Introduction
Pour la deuxième condition, nous devons d’abord reclasser la couche `buildg` de manière à ce que le résultat soit une carte booléenne avec True (1) pour l’industrie, la mine et la décharge et False (0) pour les autres classes. Ensuite, nous devons calculer la distance aux pixels vrais (True) et enfin calculer les pixels qui sont à plus de 300 m de l’industrie, de la mine et de la décharge.

## 4.2 Créer un raster booléen avec True pour l’industrie, la mine et la mise en décharge, et False pour d’autres bâtiments

Reclassons d’abord `buildg.map`. Pour les reclassifications, nous utilisons ce que l’on appelle des *tables de choix*.  Ces tables de choix peuvent être utilisées par PCRaster avec les [opérations de `choix...`](https://pcraster.geo.uu.nl/pcraster/4.3.0/documentation/pcraster_manual/sphinx/op_lookup.html). Pour ces opérations, vous devez connaître le type de données de la sortie et utiliser l’une des opérations suivantes : 


| type de donnée de sortie | opération |
|:-------:|:-------:|
| boolean | `lookupboolean` |
| nominal | `lookupnominal` |
| ordinal | `lookupordinal` |
| scalar | `lookupscalar` |
| directional | `lookupdirectional` |
| ldd | `lookupldd` |

La syntaxe génerale est:
`Result = lookup<datatype>(lookuptable,input map)`

Dans notre cas, nous devons reclasser `buildg.map` en un résultat booléen, de sorte que l’opération soit `lookupboolean`. 

Dans ce cas, le tableau doit être un fichier texte avec le contenu suivant:

`0 0
 1 0
 2 0
 3 1
 4 1
 5 1`

Vous pouvez préparer ce fichier texte dans le notebook ou un autre éditeur de texte brut. Vous pouvez lui donner n’importe quelle extension, mais ici nous allons utiliser `.tbl` pour qu’il soit clair qu’il s’agit d’un tableau. Ici, nous allons utiliser celui fourni dans le dossier `data`: `industry.tbl`.

Nous pouvons également reclasser des plages de valeurs et utiliser plusieurs cartes d’entrée. Nous y reviendrons plus tard.

Maintenant, nous pouvons utiliser l’opération `lookupboolean`. 

Pour chaque nouvelle page de notebook Jupyter, nous devons d’abord importer les bibliothèques nécessaires. Ici, nous importons PCRaster.

In [ ]:
from pcraster import *

Maintenant, nous définissons le dossier de données.

In [ ]:
os.chdir("./data")

Maintenant, nous devons lire `buildg.map` à partir du disque. Dans chaque page de notebook Jupyter, nous devons redéfinir les variables.

In [ ]:
Buildings = readmap("buildg.map")

Appliquons maintenant la table de choix.

In [ ]:
Industry = lookupboolean("industry.tbl", Buildings)

Visualisez le résultat.

## 4.3 Créer des zones de 300 mètres autour de l’industrie, de la mine et des sites de décharge
Maintenant que nous avons les pixels de l’industrie, nous pouvons créer une carte booléenne avec ‘True’ pour les pixels qui sont à au moins 300 m de l’industrie et False pour les pixels qui sont plus proches de 300 m de l’industrie.

Auparavant, nous utilisions l’opération `spreadmaxzone`. Il n’y a cependant pas d’opération `spreadminzone`. 
Il existe différentes façons de résoudre ce problème: 
1. Nous pouvons utiliser `spreadminzone`  et inverser le résultat en utilisant l’opérateur NOT `~`.
2. Nous pouvons calculer toutes les distances avec l’opération `spread` et utiliser l’opérateur `>=` pour créer la carte booléenne requise.
3. Nous pouvons utiliser une table de choix reclassant les plages. 

La méthode 1 est la plus rapide. Essayons d’abord cela.

In [ ]:
IndustryMax300m = spreadmaxzone(Industry, 0, 1, 300)
IndustryMin300m = ~ IndustryMax300m

Visualisez le résultat.

Regardons aussi les autres méthodes et voyons si elles donnent le même résultat.

Calculer les distances jusqu’à l’industrie:

In [ ]:
IndustryDistance = spread(Industry,0,1)

Créez la condition booléenne:

In [ ]:
IndustryMin300m_2 = IndustryDistance >= 300

Visualisez le résultat.

Pour la troisième méthode, nous allons utiliser une table de choix avec des plages. Dans le fichier texte, la première colonne contient les plages et la deuxième colonne les valeurs de sortie. `[` et `]` sont utilisés pour inclure la valeur dans la plage et `<` et `>` sont utilisés pour exclure la valeur de la plage.

Alors ici, notre fichier texte ressemble à:

`<,300> 0
[300,> 1`

Où la première ligne signifie: toutes les valeurs < 300 obtiennent un zéro booléen. La deuxième ligne signifie que toutes les valeurs > = 300 obtiennent booléenne 1.

Nous allons reclasser `IndustryDistance` en utilisant la table de choix `industry300m.tbl` qui se trouve dans le dossier `data`. Écrivez le code ci-dessous et écrivez le résultat dans `IndustryMin300m_3`:

Visualisez le résultat.

Vous pouvez voir que les résultats sont les mêmes. 
Dans les SIG, vous obtenez souvent le même résultat de différentes manières. Vous devez trouver l’itinéraire le plus efficace vers le résultat souhaité.

Sauvegarder le résultat comme `ind300m.map`.

<!--NAVIGATION-->
< [Condition 1](Condition_1.ipynb) | [Contenus](Contents.ipynb) | [Condition 3](Condition_3.ipynb) >